# Context manager - SafeFileEdit

Navrhněte třídu `SafeFileEdit`, která umožní otevřít soubor k zápisu, ale ve skutečnosti vyrobí dočasnou kopii, do které data zapíše a teprve na konci původní soubor přepíše. Třídu implementujte jako context manager.

In [15]:
import os
import shutil
from pathlib import Path
from typing import IO

class SafeFileEdit:
    def __init__(self, filepath):
        self.filepath = Path(filepath)
        self.tempfile_path = self.filepath.with_stem(self.filepath.stem + "tmp")
        self.handle = None

    def __enter__(self) -> IO:
        if self.filepath.exists():
            shutil.copy(self.filepath, self.tempfile_path)
        else:
            self.tempfile_path.touch()


        self.handle = open(self.tempfile_path, 'r+')
        return self.handle

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is None:
            self.handle.close()
            with open(self.tempfile_path, 'rb') as src, open(self.filepath, 'wb') as dst:
                data = src.read()
                print(data)
                dst.write(data)
        if self.tempfile_path.exists():
            os.remove(self.tempfile_path)


In [21]:
with SafeFileEdit("pokus.txt") as file:
    file.write("pokus")

b'pokuspokuspokus'
